In [ ]:
import pickle
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch, compounding
from spacy import load, displacy

In [ ]:
# load datasets

with open('ner_spacy_fmt_datasets.pickle', 'rb') as f:
    ner_spacy_fmt_datasets = pickle.load(f)

In [ ]:
nlp=spacy.blank("id")

In [ ]:
nlp.add_pipe('ner')

In [ ]:
nlp.begin_training()

In [ ]:
import random
from spacy.util import minibatch, compounding

In [ ]:
ner=nlp.get_pipe("ner")

In [ ]:
for _, annotations in ner_spacy_fmt_datasets:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        break

In [ ]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(ner_spacy_fmt_datasets)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(ner_spacy_fmt_datasets, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        examples = []
        for text, annotation in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            examples.append(example)

        nlp.update(
            examples,  # batch of Example objects
            drop=0.5,  # dropout - make it harder to memorize data
            losses=losses,
    )

    print("Losses at iteration {}".format(iteration), losses)

Losses at iteration 0 {'ner': 7180.294366765355}
Losses at iteration 1 {'ner': 4703.125468018627}
Losses at iteration 2 {'ner': 4171.913178737235}
Losses at iteration 3 {'ner': 3686.6920080995696}
Losses at iteration 4 {'ner': 3379.5960049354503}
Losses at iteration 5 {'ner': 3091.7377574625853}
Losses at iteration 6 {'ner': 2882.470451522823}
Losses at iteration 7 {'ner': 2649.7714378592927}
Losses at iteration 8 {'ner': 2643.197620865898}
Losses at iteration 9 {'ner': 2418.008992693691}
Losses at iteration 10 {'ner': 2304.91342083017}
Losses at iteration 11 {'ner': 2201.864707969879}
Losses at iteration 12 {'ner': 2212.8568426526035}
Losses at iteration 13 {'ner': 2052.647264933773}
Losses at iteration 14 {'ner': 1975.9147579493213}
Losses at iteration 15 {'ner': 1949.092075545174}
Losses at iteration 16 {'ner': 1880.0976376966998}
Losses at iteration 17 {'ner': 1818.9545578863658}
Losses at iteration 18 {'ner': 1780.5035346987793}
Losses at iteration 19 {'ner': 1681.8167868985975}
L

In [ ]:
# test
doc = nlp("jokowi makan bersama, Ganjar Pranowo di gedung MPR di New York")
print(doc.ents)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

(jokowi, Ganjar Pranowo, New York)
Entities [('jokowi', 'PERSON'), ('Ganjar Pranowo', 'PERSON'), ('New York', 'LOCATION')]


In [ ]:
# save model
from pathlib import Path

output_dir = Path('nlp_id_checkpoint_2020_10_26')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to nlp_id_checkpoint_2020_10_26


In [ ]:
# load existing model
output_dir = 'nlp_id_checkpoint_2020_10_26'
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)

Loading from nlp_id_checkpoint_2020_10_26


In [ ]:
doc = nlp_updated("Kementerian Perhubungan tidak mewajibkan rapid test COVID-19 untuk perjalanan darat lintas daerah, kecuali untuk tujuan Bali. Termasuk, dalam periode cuti bersama." )
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Bali', 'LOCATION')]


In [ ]:
displacy.render(doc, style="ent")

In [ ]:
doc = nlp_updated("Anggota Komisi IV Dewan Perwakilan Rakyat Daerah DPRD kabupaten situbondo dari Fraksi Gerindra. Joko Widodo dan Ganjar Pranowo" )
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Anggota Komisi IV Dewan Perwakilan Rakyat Daerah DPRD', 'PERSON'), ('Fraksi Gerindra', 'ORGANIZATION'), ('Joko Widodo', 'PERSON'), ('Ganjar Pranowo', 'PERSON')]


In [ ]:
displacy.render(doc, style="ent")

In [ ]:
!zip -r /content/hasiltraining.zip /content/nlp_id_checkpoint_2020_10_26

updating: content/nlp_id_checkpoint_2020_10_26/ (stored 0%)
updating: content/nlp_id_checkpoint_2020_10_26/meta.json (deflated 49%)
updating: content/nlp_id_checkpoint_2020_10_26/ner/ (stored 0%)
updating: content/nlp_id_checkpoint_2020_10_26/ner/model (deflated 7%)
updating: content/nlp_id_checkpoint_2020_10_26/ner/cfg (deflated 33%)
updating: content/nlp_id_checkpoint_2020_10_26/ner/moves (deflated 66%)
updating: content/nlp_id_checkpoint_2020_10_26/vocab/ (stored 0%)
updating: content/nlp_id_checkpoint_2020_10_26/vocab/vectors (deflated 45%)
updating: content/nlp_id_checkpoint_2020_10_26/vocab/lookups.bin (stored 0%)
updating: content/nlp_id_checkpoint_2020_10_26/vocab/key2row (stored 0%)
updating: content/nlp_id_checkpoint_2020_10_26/vocab/vectors.cfg (stored 0%)
updating: content/nlp_id_checkpoint_2020_10_26/vocab/strings.json (deflated 79%)
updating: content/nlp_id_checkpoint_2020_10_26/tokenizer (deflated 75%)
updating: content/nlp_id_checkpoint_2020_10_26/config.cfg (deflated 5